In [1]:
# Importing required dependencies
from pyspark.sql import Row
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
from pyspark.sql.types import StringType
from pyspark.sql import types

In [2]:
sc

# RDD Operations

In [3]:
# Extracting data from the textFile
sourcePath = "/Users/pravinkumar/Documents/Spark/testData/weblogs"
wholeTextRDD = sc.wholeTextFiles(sourcePath)
#for i in wholeTextRDD.take(10): print(i)

In [4]:
# Extracting data from the textFile
sourcePath = "/Users/pravinkumar/Documents/Spark/testData/cards/"
textRDD = sc.textFile(sourcePath + "deckofcards.txt")

for i in textRDD.take(10): print(i)
print("*"*75)    

def fun(x):
    spade = 0
    club = 0
    diamond = 0
    heart = 0
    for i in x:
        if (i == "SPADE"):
            spade += 1
        elif (i == "CLUB"):
            club += 1
        elif (i == "DIAMOND"):
            diamond += 1
        elif (i == "HEART"):
            heart += 1
    return ({"SPADE": spade, "CLUB": club, "DIAMOND": diamond, "HEART": heart})

# Using GroupByKey        
textRDDGroupBy = textRDD.map(lambda rec: rec.split("|")).map(lambda rec: (rec[0], (rec[1]))).\
groupByKey().mapValues(fun).flatMapValues(lambda rec: [(i, rec[i]) for i in rec.keys()])

for i in textRDDGroupBy.take(10): print(i)
print("*"*75)    

# Using ReduceByKey
textRDDReduceBy = textRDD.map(lambda rec: rec.split("|")).map(lambda rec: ((rec[0], rec[1]), 1)).\
reduceByKey(lambda agg,value: agg + value)
for i in textRDDReduceBy.take(10): print(i)
print("*"*75)

# Using AggregateByKey
seqOp = (lambda agg, val: agg + val)
combOp = (lambda agg, val: agg + val)
textRDDAggBy = textRDD.map(lambda rec: rec.split("|")).map(lambda rec: ((rec[0], rec[1]), 1)).\
aggregateByKey(0, seqOp, combOp)                                                           
for i in textRDDAggBy.take(10): print(i)
                                                           
# Using ReduceByKey with two values
reduceByForFun = textRDD.map(lambda rec: rec.split("|")).map(lambda rec: ((rec[0], rec[1]), ("1",1.0))).\
reduceByKey(lambda agg,value: (agg[0]+"(0_0)"+value[0], agg[1]+(value[1]*2.2)))

# Using AggregateByKey with two values
seqOp = (lambda agg, val: (agg[0] + 1, agg[1] + val[0], agg[2] + val[1]))
combOp = (lambda agg, val: (agg[0] + val[0], agg[1] + "(0_0)"+ val[1], agg[2] / val[2]))
aggregateByForFun = textRDD.map(lambda rec: rec.split("|")).map(lambda rec: ((rec[0], rec[1]), ("1",1.0))).\
aggregateByKey((0, "", 0.0), seqOp, combOp)

for i in aggregateByForFun.take(10): print(i)

BLACK|SPADE|2
BLACK|SPADE|3
BLACK|SPADE|4
BLACK|SPADE|5
BLACK|SPADE|6
BLACK|SPADE|7
BLACK|SPADE|8
BLACK|SPADE|9
BLACK|SPADE|10
BLACK|SPADE|J
***************************************************************************
('BLACK', ('CLUB', 13))
('BLACK', ('DIAMOND', 0))
('BLACK', ('SPADE', 13))
('BLACK', ('HEART', 0))
('RED', ('CLUB', 0))
('RED', ('DIAMOND', 13))
('RED', ('SPADE', 0))
('RED', ('HEART', 13))
***************************************************************************
(('BLACK', 'CLUB'), 13)
(('BLACK', 'SPADE'), 13)
(('RED', 'HEART'), 13)
(('RED', 'DIAMOND'), 13)
***************************************************************************
(('BLACK', 'CLUB'), 13)
(('BLACK', 'SPADE'), 13)
(('RED', 'HEART'), 13)
(('RED', 'DIAMOND'), 13)
(('BLACK', 'CLUB'), (13, '1111111111111', 13.0))
(('BLACK', 'SPADE'), (13, '1111111111111', 13.0))
(('RED', 'HEART'), (13, '1111111111111', 13.0))
(('RED', 'DIAMOND'), (13, '1111111111111', 13.0))


In [5]:
head = textRDD.take(10)
for i in head:print(i)

BLACK|SPADE|2
BLACK|SPADE|3
BLACK|SPADE|4
BLACK|SPADE|5
BLACK|SPADE|6
BLACK|SPADE|7
BLACK|SPADE|8
BLACK|SPADE|9
BLACK|SPADE|10
BLACK|SPADE|J


In [6]:
# Word count in the file
wordCountRDD = textRDD.flatMap(lambda rec: rec.split("|")).map(lambda rec: (rec, 1)).\
reduceByKey(lambda agg, value: agg + value)
for i in wordCountRDD.take(10): print(i)
    
wordCountRDD = textRDD.flatMap(lambda rec: rec.split("|")).map(lambda rec: (rec, 1)).\
countByKey()

print(wordCountRDD)
for k, v in wordCountRDD.items():
    print("'{}' : {}".format(k, v))
print("*"*75)

('10', 4)
('4', 4)
('CLUB', 13)
('SPADE', 13)
('J', 4)
('BLACK', 26)
('K', 4)
('RED', 26)
('8', 4)
('9', 4)
defaultdict(<class 'int'>, {'10': 4, 'CLUB': 13, '5': 4, 'SPADE': 13, '3': 4, '2': 4, 'BLACK': 26, '7': 4, '8': 4, '6': 4, '4': 4, '9': 4, 'DIAMOND': 13, 'K': 4, 'Q': 4, 'J': 4, 'RED': 26, 'HEART': 13, 'A': 4})
'10' : 4
'CLUB' : 13
'5' : 4
'SPADE' : 13
'3' : 4
'2' : 4
'BLACK' : 26
'7' : 4
'8' : 4
'6' : 4
'4' : 4
'9' : 4
'DIAMOND' : 13
'K' : 4
'Q' : 4
'J' : 4
'RED' : 26
'HEART' : 13
'A' : 4
***************************************************************************


In [7]:
# Performing Join operation
for i in textRDD.take(10): print(i)
leftRDD = textRDD.map(lambda rec: rec.split("|")).map(lambda rec: (rec[1], rec[0]))
rightRDD = textRDD.map(lambda rec: rec.split("|")).map(lambda rec: (rec[1], rec[2]))

count = sc.accumulator(0)

resultJoinRDD = leftRDD.join(rightRDD)

def f(x):
    global count
    count += 1

resultJoinRDD.foreach(f)
    
print(count)

BLACK|SPADE|2
BLACK|SPADE|3
BLACK|SPADE|4
BLACK|SPADE|5
BLACK|SPADE|6
BLACK|SPADE|7
BLACK|SPADE|8
BLACK|SPADE|9
BLACK|SPADE|10
BLACK|SPADE|J
676


# RDD to DF conversion

In [8]:
# Converting RDD to DF  

# Method 1
schema = StructType([
    StructField("color", types.StringType(), True),
    StructField("shape", types.StringType(), True),
    StructField("number", types.StringType(), True)
])
textRDDToDF01 = textRDD.map(lambda rec: rec.split("|"))   
textDF = sqlContext.createDataFrame(textRDDToDF01, schema)
textDF.show()
print("*"*75)        

# Method 2
textRDDToDF02 = textRDD.map(lambda rec: rec.split("|")).map(lambda rec: Row(color= rec[0], shape= rec[1], \
                                                                            number= rec[2])).toDF()
#for i in textDF.take(10): print(i)
print(textRDDToDF02.printSchema())
textRDDToDF02.show()
print("*"*75)

# Getting only the required columns
textRDDToDF02.select("color", "number").show()



+-----+-----+------+
|color|shape|number|
+-----+-----+------+
|BLACK|SPADE|     2|
|BLACK|SPADE|     3|
|BLACK|SPADE|     4|
|BLACK|SPADE|     5|
|BLACK|SPADE|     6|
|BLACK|SPADE|     7|
|BLACK|SPADE|     8|
|BLACK|SPADE|     9|
|BLACK|SPADE|    10|
|BLACK|SPADE|     J|
|BLACK|SPADE|     Q|
|BLACK|SPADE|     K|
|BLACK|SPADE|     A|
|BLACK| CLUB|     2|
|BLACK| CLUB|     3|
|BLACK| CLUB|     4|
|BLACK| CLUB|     5|
|BLACK| CLUB|     6|
|BLACK| CLUB|     7|
|BLACK| CLUB|     8|
+-----+-----+------+
only showing top 20 rows

***************************************************************************
root
 |-- color: string (nullable = true)
 |-- number: string (nullable = true)
 |-- shape: string (nullable = true)

None
+-----+------+-----+
|color|number|shape|
+-----+------+-----+
|BLACK|     2|SPADE|
|BLACK|     3|SPADE|
|BLACK|     4|SPADE|
|BLACK|     5|SPADE|
|BLACK|     6|SPADE|
|BLACK|     7|SPADE|
|BLACK|     8|SPADE|
|BLACK|     9|SPADE|
|BLACK|    10|SPADE|
|BLACK|     J|SPADE

# DF Operations

In [9]:
# Getting the column details
print(textDF.columns)
# getting count of records
print(textDF.count())

['color', 'shape', 'number']
52


In [10]:
# Grouping operation on DF
textDF.groupBy(textDF.color).agg({"*": "count"}).show()

+-----+--------+
|color|count(1)|
+-----+--------+
|  RED|      26|
|BLACK|      26|
+-----+--------+



In [11]:
# Filtering operation on DF
textDF.filter(textDF.number == "2").show()

+-----+-------+------+
|color|  shape|number|
+-----+-------+------+
|BLACK|  SPADE|     2|
|BLACK|   CLUB|     2|
|  RED|DIAMOND|     2|
|  RED|  HEART|     2|
+-----+-------+------+



In [12]:
# Joining two tables on a particular column
textLeft = textDF.select("color", "shape")
textRight = textDF.select("shape", "number")

resultJoin = textLeft.join(textRight, textLeft.shape == textRight.shape).drop(textRight.shape)
# getting count of records after joining
# its maching each shape of textLeft to each shape of textRight which results in 52*13, so be carfull with this operation
print(resultJoin.count())
resultJoin.show()

676
+-----+-------+------+
|color|  shape|number|
+-----+-------+------+
|  RED|DIAMOND|     2|
|  RED|DIAMOND|     3|
|  RED|DIAMOND|     4|
|  RED|DIAMOND|     5|
|  RED|DIAMOND|     6|
|  RED|DIAMOND|     7|
|  RED|DIAMOND|     8|
|  RED|DIAMOND|     9|
|  RED|DIAMOND|    10|
|  RED|DIAMOND|     J|
|  RED|DIAMOND|     Q|
|  RED|DIAMOND|     K|
|  RED|DIAMOND|     A|
|  RED|DIAMOND|     2|
|  RED|DIAMOND|     3|
|  RED|DIAMOND|     4|
|  RED|DIAMOND|     5|
|  RED|DIAMOND|     6|
|  RED|DIAMOND|     7|
|  RED|DIAMOND|     8|
+-----+-------+------+
only showing top 20 rows



In [13]:
# Registering DF as a Temp Table and performing queries
textDF.registerTempTable("data")
sqlContext.getConf("spark.sql.shuffle.partitions", "100")

query = sqlContext.sql("select * from data")
query.show()

# Converting DF to RDD and printing the data
for i in query.rdd.map(list).take(10): print(i)

+-----+-----+------+
|color|shape|number|
+-----+-----+------+
|BLACK|SPADE|     2|
|BLACK|SPADE|     3|
|BLACK|SPADE|     4|
|BLACK|SPADE|     5|
|BLACK|SPADE|     6|
|BLACK|SPADE|     7|
|BLACK|SPADE|     8|
|BLACK|SPADE|     9|
|BLACK|SPADE|    10|
|BLACK|SPADE|     J|
|BLACK|SPADE|     Q|
|BLACK|SPADE|     K|
|BLACK|SPADE|     A|
|BLACK| CLUB|     2|
|BLACK| CLUB|     3|
|BLACK| CLUB|     4|
|BLACK| CLUB|     5|
|BLACK| CLUB|     6|
|BLACK| CLUB|     7|
|BLACK| CLUB|     8|
+-----+-----+------+
only showing top 20 rows

['BLACK', 'SPADE', '2']
['BLACK', 'SPADE', '3']
['BLACK', 'SPADE', '4']
['BLACK', 'SPADE', '5']
['BLACK', 'SPADE', '6']
['BLACK', 'SPADE', '7']
['BLACK', 'SPADE', '8']
['BLACK', 'SPADE', '9']
['BLACK', 'SPADE', '10']
['BLACK', 'SPADE', 'J']


# Importing & Exporting in Different File Formats

In [16]:
rdd = sc.parallelize(range(1, 4)).map(lambda x: (x, "a" * x)) 
rdd.saveAsSequenceFile("/Users/pravinkumar/Documents/Spark/Results/path/to/file")
sorted(sc.sequenceFile("/Users/pravinkumar/Documents/Spark/Results/path/to/file", "org.apache.hadoop.io.IntWritable", "org.apache.hadoop.io.Text").collect())

[(1, 'a'), (2, 'aa'), (3, 'aaa')]

In [17]:
# Demonstrating saveAsSequenceFile and reading SequenceFile

# Saving the file in SequenceFile Format
Destinationpath = "/Users/pravinkumar/Documents/Spark/Results/"
textRDD.map(lambda x: (None, x)).saveAsSequenceFile(Destinationpath + "deckofcardsSeq")

# Reading the file in SequenceFile Format
SeqRDD = sc.sequenceFile(Destinationpath + "deckofcardsSeq")

for i in SeqRDD.take(10): print(i)
print("*"*75)   

# Demonstrating Saving and Reading SequenceFile Using saveAsNewAPIHadoopFile

# Saving the file in SequenceFile Format
textRDD.map(lambda x: (None, x)).saveAsNewAPIHadoopFile(Destinationpath + "deckofcardsSeqWithNewAPI","org.apache.hadoop.mapreduce.lib.output.SequenceFileOutputFormat",\
                               keyClass=None,valueClass="org.apache.hadoop.io.Text")

# Reading the file in SequenceFile Format
largedeckSeqWithNewAPI = sc.sequenceFile(Destinationpath + "deckofcardsSeqWithNewAPI")

for i in largedeckSeqWithNewAPI.take(10): print(i)
print("*"*75)

(None, 'BLACK|SPADE|2')
(None, 'BLACK|SPADE|3')
(None, 'BLACK|SPADE|4')
(None, 'BLACK|SPADE|5')
(None, 'BLACK|SPADE|6')
(None, 'BLACK|SPADE|7')
(None, 'BLACK|SPADE|8')
(None, 'BLACK|SPADE|9')
(None, 'BLACK|SPADE|10')
(None, 'BLACK|SPADE|J')
***************************************************************************
(None, 'BLACK|SPADE|2')
(None, 'BLACK|SPADE|3')
(None, 'BLACK|SPADE|4')
(None, 'BLACK|SPADE|5')
(None, 'BLACK|SPADE|6')
(None, 'BLACK|SPADE|7')
(None, 'BLACK|SPADE|8')
(None, 'BLACK|SPADE|9')
(None, 'BLACK|SPADE|10')
(None, 'BLACK|SPADE|J')
***************************************************************************


In [18]:
# Writing the file to CSV, JSON, ORC, Parquet from DataFrame

textDF.write.csv(Destinationpath + "deckofcardsCSV")
textDF.write.json(Destinationpath + "deckofcardsJSON")
textDF.write.orc(Destinationpath + "deckofcardsORC")
textDF.write.parquet(Destinationpath + "deckofcardsParquet")
textDF.write.format("json").save(Destinationpath + "deckofcardsJSONThroughFormat")
textDF.write.save(Destinationpath + "deckofcardsParquetThroughFormat", format = "parquet")


In [19]:
Destinationpath = "/Users/pravinkumar/Documents/Spark/Results/"

# Loading file in CSV format
largedeckCSVDF = sqlContext.read.csv(Destinationpath + "deckofcardsCSV")
largedeckCSVDF = sqlContext.read.load(Destinationpath + "deckofcardsCSV", format = "csv")
largedeckCSVDF.printSchema()
largedeckCSVDF.show()
print("*"*75)

# Loading file in JSON format
largedeckJSONDF = sqlContext.read.json(Destinationpath + "deckofcardsJSON")
largedeckJSONDF.printSchema()
largedeckJSONDF.show()
print("*"*75)

# Converting DF to RDD -> list(data)
largedeckJSONDF = sqlContext.read.json(Destinationpath + "deckofcardsJSON").rdd.map(list)
for i in largedeckJSONDF.take(10): print(i)
print("*"*75)

# Loading file in ORC format
largedeckORCDF = sqlContext.read.orc(Destinationpath + "deckofcardsORC")
largedeckORCDF.printSchema()
largedeckORCDF.show()
print("*"*75)

# Loading file in Parquet format
largedeckParquetDF = sqlContext.read.parquet(Destinationpath + "deckofcardsParquet")
largedeckParquetDF.printSchema()
largedeckParquetDF.show()
print("*"*75)



root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)

+-----+-----+---+
|  _c0|  _c1|_c2|
+-----+-----+---+
|BLACK|SPADE|  2|
|BLACK|SPADE|  3|
|BLACK|SPADE|  4|
|BLACK|SPADE|  5|
|BLACK|SPADE|  6|
|BLACK|SPADE|  7|
|BLACK|SPADE|  8|
|BLACK|SPADE|  9|
|BLACK|SPADE| 10|
|BLACK|SPADE|  J|
|BLACK|SPADE|  Q|
|BLACK|SPADE|  K|
|BLACK|SPADE|  A|
|BLACK| CLUB|  2|
|BLACK| CLUB|  3|
|BLACK| CLUB|  4|
|BLACK| CLUB|  5|
|BLACK| CLUB|  6|
|BLACK| CLUB|  7|
|BLACK| CLUB|  8|
+-----+-----+---+
only showing top 20 rows

***************************************************************************
root
 |-- color: string (nullable = true)
 |-- number: string (nullable = true)
 |-- shape: string (nullable = true)

+-----+------+-----+
|color|number|shape|
+-----+------+-----+
|BLACK|     2|SPADE|
|BLACK|     3|SPADE|
|BLACK|     4|SPADE|
|BLACK|     5|SPADE|
|BLACK|     6|SPADE|
|BLACK|     7|SPADE|
|BLACK|     8|SPADE|
|BLACK|     9|SPADE|
|BLACK

In [20]:
# import needed for the .avro method to be added


# The Avro records get converted to Spark types, filtered, and
# then written back out as Avro records
textDF.write.format("com.databricks.spark.avro").save(Destinationpath + "deckofcardsAVRO")
largedeckAVRODF = spark.read.format("com.databricks.spark.avro").load(Destinationpath + "deckofcardsAVRO")

largedeckAVRODF.printSchema()
largedeckAVRODF.show()


AnalysisException: 'Failed to find data source: com.databricks.spark.avro. Please find an Avro package at http://spark.apache.org/third-party-projects.html;'